In [1]:
import numpy as np
import random
import tensorflow as tf
import matplotlib.pyplot as plt
import grpc

import simulation_service_pb2 as v1
import simulation_service_pb2_grpc as v1GRPC

# Helper Functions

In [3]:
# For generating a random action
actions = [
    ["MOVE", "UP"],
    ["MOVE", "DOWN"],
    ["MOVE", "LEFT"],
    ["MOVE", "RIGHT"],
    ["CONSUME", "UP"],
    ["CONSUME", "DOWN"],
    ["CONSUME", "LEFT"],
    ["CONSUME", "RIGHT"],
    ["WAIT", "DOWN"]
]

def randomAction():
    return random.randint(0, 8)
def cellStringToNumber(s):
    if (s == "EMPTY"):
        return 0
    elif (s == "FOOD"):
        return 1
    else:
        return 0

# Connect to our servers

In [4]:
channel = grpc.insecure_channel('35.238.189.154:9090')
stub = v1GRPC.SimulationServiceStub(channel)
metadata = [('auth-secret', '<your-secret-key>')]
api="v1"

In [ ]:
observations = stub.CreateRemoteModel(v1.CreateRemoteModelRequest(api=api, name="My Model"), metadata=metadata)
# NOTE: This try-catch is EXTREMELY important. 
# If you don't put this here, you won't be able to remove your model!
# With it here you can simply stop this cell and it will be disconnected from our servers
try:
    for obsv in observations:
        # This is example code, put your model execution here!
        if (obsv.alive == False):
            continue
        action = actions[randomAction()]
        actionId = action[0]
        actionDir = action[1]
        action = v1.Action(id=actionId, direction=actionDir)
        actionRes = stub.ExecuteAgentAction(v1.ExecuteAgentActionRequest(api=api, id=obsv.id, action=action), metadata=metadata)
except KeyboardInterrupt:
    observations.cancel()